In [1]:
1*2

2

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import gpytorch
from gpytorch import settings
from gpytorch.constraints import Positive
from gpytorch.constraints import LessThan
import itertools
from itertools import combinations

n_devices = torch.cuda.device_count()
output_device = torch.device('cuda:0')


def calc_L_polynomial_coeffs():
        lambdas = np.array([q**k for k in range(l+1)])
        s = l + 1
        B = np.zeros((s, s))
        idx = np.arange(s)
        for k in idx:
            k_idx = idx != k
            k_lambdas = lambdas[k_idx]
            norm_factor = 1 / np.prod(k_lambdas - lambdas[k])

            for power in idx:
                p = np.sum([np.product(v) for v in combinations(k_lambdas, l - power)])
                B[power, k] = norm_factor * (-1) ** (power) * p
        return(B)

class CustomKernel(gpytorch.kernels.kernel.Kernel):
    is_stationary = True
    def __init__(self, alpha, l,train_p=True,
                log_lda_prior=None, log_lda_constraint=None, 
                log_p_prior=None, log_p_constraint=None,
                **kwargs):
        super().__init__(**kwargs)
        
        self.odds = torch.nn.Parameter(torch.tensor([q**t/(1 - q**t) for t in range(1, l+1)]), requires_grad=False)
        self.scaling_factors = torch.tensor([(1 - q**t)**l for t in range(l+1)])
        self.scaling_factors[0] = 1
        self.scaling_factors = torch.nn.Parameter(self.scaling_factors, requires_grad=False)
        
        self.coeffs = torch.tensor(calc_L_polynomial_coeffs(), dtype=torch.float32)
        self.coeffs = torch.nn.Parameter(self.coeffs, requires_grad=False)

        # register the raw parameter
        self.register_parameter(
          name='raw_log_p', 
          parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, l, alpha), requires_grad=train_p)
        )
        self.register_parameter(
          name='raw_log_lda', 
          parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, l+1))
        )

        # set the parameter constraint to be positive, when nothing is specified
        if log_lda_constraint is None:
          log_lda_constraint = LessThan(upper_bound=0.)

        if log_p_constraint is None:
          log_p_constraint = LessThan(upper_bound=0.)

        # register the constraint
        self.register_constraint("raw_log_lda", log_lda_constraint)
        self.register_constraint("raw_log_p", log_p_constraint)

    @property
    def log_lda(self):
      return self.raw_log_lda_constraint.transform(self.raw_log_lda)

    @property
    def log_p(self):
      return self.raw_log_p_constraint.transform(self.raw_log_p)

    @log_lda.setter
    def log_lda(self, value):
      return self._set_log_lda(value)

    @log_p.setter
    def log_p(self, value):
      return self._set_log_p(value)


    def forward(self, x1, x2, diag=False, **params):
        # construct masks used for calculate rates
        masks = torch.mul(torch.unsqueeze(x1, 1), torch.unsqueeze(x2, 0))
        ps = torch.softmax(self.log_p, axis=1)
        
        pi = x2*(torch.flatten(ps))
        pi[pi==0.] = 1
        pi = torch.prod(pi, 1)
        Dpi = torch.diag(pi)        

        rates = self.odds.unsqueeze(1).unsqueeze(-1)  + torch.unsqueeze(ps, 0)
        rates = rates/ps
        rates = torch.flatten(rates, start_dim=1)
        log_rates = torch.log(rates)
        
        out = torch.mul(masks.unsqueeze(2), log_rates)
        out = torch.flatten(out, start_dim=3)

        powers_nz = torch.exp(torch.sum(out, -1))
        power_0 = F.relu(torch.sum(masks, -1) - l + 1).matmul(torch.linalg.inv(Dpi))
        powers = torch.cat([power_0.unsqueeze(-1), powers_nz], dim=-1)
        powers = powers*self.scaling_factors
        
        weights = torch.matmul(self.coeffs, torch.exp(self.log_lda))
        
        k = torch.sum(torch.mul(powers, weights), -1)
                
        return k

    
class SkewVCModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, ker):
        super(SkewVCModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        base_covar_module = ker


        self.covar_module = gpytorch.kernels.MultiDeviceKernel(
            base_covar_module, device_ids=range(n_devices),
            output_device=output_device
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


# import data
alpha = 4
l = 8
q = 0.7

!wget https://raw.githubusercontent.com/davidmccandlish/vcregression/master/vcregression/data/Smn1/smn1data.csv
    
import pandas as pd
dat = pd.read_csv("Smn1Data.txt", header=None)
dat = dat.rename(columns={0:"seq", 1:"psi", 2:"std", 3:"gene"})
dat['seq']=[seq[:3] + seq[4:] for seq in dat['seq']]

from collections import OrderedDict
IUPAC_VOCAB_ = OrderedDict([
    ("A", 0),
    ("U", 1),
    ("C", 2),
    ("G", 3)])

def tokenize(seq):
    return [IUPAC_VOCAB_[char] for char in seq]

seqs = [tokenize(seq) for seq in dat.seq]
seqs = torch.tensor(seqs).to(output_device)
seqs1h = torch.flatten(torch.tensor(F.one_hot(seqs),dtype=torch.float32), start_dim=1).to(output_device)
y = torch.tensor(dat.psi, dtype=torch.float32).to(output_device)


# train test data
import random
train_size = 15000
train_ids = random.sample(range(len(seqs1h)), train_size)
test_ids = random.sample(list(set(range((len(seqs1h)))).difference(train_ids)), dat.shape[0] - train_size)

train_x, test_x = seqs1h[train_ids], seqs1h[test_ids]
train_y, test_y = y[train_ids], y[test_ids]

# Define likelihood
train_noise = np.array(dat['std'].iloc[train_ids])**2
train_noise = torch.tensor(train_noise, dtype=torch.float32).to(output_device)
likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=train_noise, 
                                                               learn_additional_noise=True).to(output_device)
# Define model
kernel = CustomKernel(alpha, l, train_p=True)
# kernel = kernel.to(output_device)
kernel.raw_log_lda = torch.nn.Parameter(torch.cat((torch.tensor([-100.]), 
                                                   -2*torch.arange(l))).to(output_device))
model = SkewVCModel(train_x, train_y, likelihood, kernel).to(output_device)

# predicting for test points using partitioning
checkpoint_size = train_x.shape[0]//2
# checkpoint_size = 0

from sklearn.metrics import r2_score
from scipy.stats import pearsonr

test_x = test_x.cuda()
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.beta_features.checkpoint_kernel(checkpoint_size):
    f_preds = model(test_x)

--2022-09-08 21:17:54--  https://raw.githubusercontent.com/davidmccandlish/vcregression/master/vcregression/data/Smn1/smn1data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1445664 (1.4M) [text/plain]
Saving to: ‘smn1data.csv.1’

100%[======================================>] 1,445,664   --.-K/s   in 0.1s    

2022-09-08 21:17:54 (9.91 MB/s) - ‘smn1data.csv.1’ saved [1445664/1445664]



/scratch/local/46964012/ipykernel_38557/1704718296.py:156: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seqs1h = torch.flatten(torch.tensor(F.one_hot(seqs),dtype=torch.float32), start_dim=1).to(output_device)


RuntimeError: CUDA out of memory. Tried to allocate 214.58 GiB (GPU 0; 79.35 GiB total capacity; 27.68 GiB already allocated; 50.41 GiB free; 27.68 GiB reserved in total by PyTorch)

In [3]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 36% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |
|  4 |  0% |  0% |
|  5 |  0% |  0% |
|  6 |  0% |  0% |
|  7 |  0% |  0% |


In [12]:
print(torch.__version__)

1.8.1


In [13]:
print(gpytorch.__version__)

1.8.1
